In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import tensorflow as tf
from tensorflow import keras
from keras import Input, Model
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten, ReLU
# import torch.optim as optim

from time import time
from scipy.stats import poisson, gamma, norm
from scipy.optimize import minimize_scalar
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
# mp.set_start_method("spawn") 
# from sddr import Sddr  # Assuming you have pyssdr installed and configured correctly
import logging
from datetime import datetime
# import torch
logging.basicConfig(level=logging.INFO)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

2025-02-16 16:43:24.504783: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-16 16:43:24.767005: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def scale_to_range(data, lower=-1, upper=1):
    return (data - np.min(data)) / (np.max(data) - np.min(data)) * (upper - lower) + lower

def build_dnn(input_dim, layer_sizes=[32, 16], activation="relu"):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(layer_sizes[0], activation=activation),
        Dense(layer_sizes[1], activation=activation),
        Dense(1, activation=None)
    ])
    model.compile(optimizer='adam', loss='mse')  # Compile the model

    return model

def save_with_var_name(var, var_name, var_type, save_path, scenario_index):
    if var_type == 'npy':
        np.save(f"{save_path}/{var_name}_{scenario_index}.npy", var)
    if var_type == 'keras':
        var.save(f"{save_path}/{var_name}_{scenario_index}.keras")
    if var_type == 'jpgs':
        images_path = f"{save_path}/{var_name}_{scenario_index}"
        os.makedirs(images_path, exist_ok=True)
        for idx, img in enumerate(var):
            img = img.convert("L")  
            img.save(f"{images_path}/{var_name}_{scenario_index}_{idx}.jpg")
    logging.info(f"Saved {var_name} to {save_path}/{var_name}_{scenario_index}.npy")

def read_with_var_name(var_name, var_type, save_path, scenario_index):
    if var_type == 'npy':
        return np.load(f"{save_path}/{var_name}_{scenario_index}.npy")
    if var_type == 'keras':
        return tf.keras.models.load_model(f"{save_path}/{var_name}_{scenario_index}.keras")
    

In [4]:
def SNR_compute(n_list, distribution_list, SNR_list, grid_size, alpha_l, beta_nl, n_rep=100):
    """
    Compute the SNR for scenarios to verify its correctness.

    Parameters:
    - n_list: List of sample sizes
    - distribution_list: List of distributions (e.g., ["poisson", "gamma", "gaussian"])
    - SNR_list: List of SNR values
    - grid_size: Size of the grid for unstructured effects
    - alpha_l: Coefficients for linear effects
    - beta_nl: Coefficients for nonlinear effects
    - n_rep: Number of repetitions for each scenario

    Returns:
    - Results of the SNR validation.
    """
    save_path = "output"
    results = []

    for n in n_list:
        for rep in range(n_rep):
            scenario_index = f"n_{n}_rep_{rep}"
            # X = read_with_var_name('X', 'npy', save_path, scenario_index)
            # linear_effects = read_with_var_name('linear_effects', 'npy', save_path, scenario_index)
            
            # Z = read_with_var_name('Z', 'npy', save_path, scenario_index)
            # nonlinear_effects = read_with_var_name('nonlinear_effects', 'npy', save_path, scenario_index)
            
            # images = read_with_var_name('images', 'npy', save_path, scenario_index)
            # unstructured_effects = read_with_var_name('unstructured_effects', 'npy', save_path, scenario_index)
            # U_k = read_with_var_name('U_k', 'npy', save_path, scenario_index)
            # psi_k = read_with_var_name('psi_k', 'npy', save_path, scenario_index)
            # b_k = read_with_var_name('b_k', 'npy', save_path, scenario_index)
            
            # try:
            # Read the effects and response data
            for dist in distribution_list:
                for snr in SNR_list:
                    dist_snr_index = f"{scenario_index}_dist_{dist}_SNR_{snr}"
                    a = read_with_var_name('a', 'npy', save_path, dist_snr_index)
                    etas = read_with_var_name('etas', 'npy', save_path, dist_snr_index)
                    # responses = read_with_var_name('responses', 'npy', save_path, dist_snr_index)
                    # print(dist_snr_index,":", etas)
                    ptp_eta = np.ptp(etas[:, 0])
                    if dist == "poisson":
                        snr_computed = (ptp_eta / np.sqrt(np.exp(etas[:, 0]))).mean()
                    # elif dist == "gamma":
                    #     snr_computed = ptp_eta / a # sigma
                    elif dist == "gaussian_homo":
                        std_eta = a
                        snr_computed = (ptp_eta / std_eta).mean()
                    elif dist == "gaussian_hetero":
                        std_eta = np.exp(etas[:, 1])
                        snr_computed = (ptp_eta / std_eta).mean()
                    else:
                        raise ValueError(f"Unsupported distribution: {dist}")
                    results.append({
                            "n": n,
                            "rep": rep,
                            "distribution": dist,
                            "SNR": snr,
                            "computed_SNR": snr_computed
                        })
    return results


In [5]:
n_list = [100]
distribution_list = ["poisson", "gaussian_homo", "gaussian_hetero"] #["poisson", "gamma", "gaussian"]
SNR_list=[1,8]
n_rep=2
grid_size = 28
# Linear effect coefficients
alpha_l = {0: [3, -1], 1: [-0.5, 6]}

# Nonlinear effect functions
def nonlinear_effect_1_z1(z1):
    return scale_to_range(3 * np.sin(6 * z1))

def nonlinear_effect_1_z2(z2):
    return scale_to_range(np.exp(5 * z2))

def nonlinear_effect_2_z1(z1):
    return scale_to_range(np.cos(8 * z1))

def nonlinear_effect_2_z2(z2):
    return scale_to_range(0.1 * np.sqrt(z2))

beta_nl = {
    0: [nonlinear_effect_1_z1, nonlinear_effect_1_z2],
    1: [nonlinear_effect_2_z1, nonlinear_effect_2_z2]
}

result = pd.DataFrame(SNR_compute(n_list, distribution_list, SNR_list, grid_size, alpha_l, beta_nl, n_rep=n_rep))

In [6]:
result

,n,rep,distribution,SNR,computed_SNR
0,100,0,poisson,1,1.000000
1,100,0,poisson,8,8.000006
2,100,0,gaussian_homo,1,1.000000
3,100,0,gaussian_homo,8,8.000012
4,100,0,gaussian_hetero,1,1.000000
5,100,0,gaussian_hetero,8,8.000008
6,100,1,poisson,1,1.000000
7,100,1,poisson,8,8.000000
8,100,1,gaussian_homo,1,1.000000
9,100,1,gaussian_homo,8,7.999985


In [ ]:
# ---------------------------
# Helper Functions for Visualization
# ---------------------------

def variance_of_difference(img1, img2):
    difference = img1 - img2
    return np.var(difference)

def mean_absolute_error(img1, img2):
    return np.mean(np.abs(img1 - img2))

def pixel_difference_count(img1, img2, threshold=0.01):
    return np.sum(np.abs(img1 - img2) > threshold)

def scale_to_range(data, lower=-1, upper=1):
    return (data - np.min(data)) / (np.max(data) - np.min(data)) * (upper - lower) + lower



def plot_linear_effects(X, linear_effects1, linear_effects2, scenario1, scenario2, K):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
    colors = plt.cm.viridis(np.linspace(0, 1, K * linear_effects1.shape[0]))

    color_idx = 0
    for k in range(K):
        for i in range(linear_effects1.shape[0]):  # Iterate over I
            axes[0].plot(X[i, :, k], linear_effects1[i, :, k], 'o', 
                         label=f'Linear Effect {k+1}, X{i+1} for low SNR', color=colors[color_idx])
            axes[1].plot(X[i, :, k], linear_effects2[i, :, k], 'o', 
                         label=f'Linear Effect {k+1}, X{i+1} for high SNR', color=colors[color_idx])
            color_idx += 1
    
    axes[0].set_title(f"Linear Effects (low SNR) - Scenario {scenario1}")
    axes[0].set_xlabel("X (scaled)")
    axes[0].set_ylabel("True Effect Value")
    axes[0].legend()

    axes[1].set_title(f"Linear Effects (high SNR) - Scenario {scenario2}")
    axes[1].set_xlabel("X (scaled)")
    axes[1].legend()

    plt.tight_layout()
    plt.show()

def plot_nonlinear_effects(Z, nonlinear_effects1, nonlinear_effects2, scenario1, scenario2, K):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
    colors = plt.cm.viridis(np.linspace(0, 1, K * nonlinear_effects1.shape[0]))

    color_idx = 0
    for k in range(K):
        for j in range(nonlinear_effects1.shape[0]):  # Iterate over J
            axes[0].plot(Z[j, :, k], nonlinear_effects1[j, :, k], 'o', 
                         label=f'Nonlinear Effect {k+1}, Z{j+1} for low SNR', color=colors[color_idx])
            axes[1].plot(Z[j, :, k], nonlinear_effects2[j, :, k], 'o', 
                         label=f'Nonlinear Effect {k+1}, Z{j+1} for high SNR', color=colors[color_idx])
            color_idx += 1
    
    axes[0].set_title(f"Nonlinear Effects (low SNR) - Scenario {scenario1}")
    axes[0].set_xlabel("Z (scaled)")
    axes[0].set_ylabel("True Effect Value")
    axes[0].legend()

    axes[1].set_title(f"Nonlinear Effects (high SNR) - Scenario {scenario2}")
    axes[1].set_xlabel("Z (scaled)")
    axes[1].legend()

    plt.tight_layout()
    plt.show()

def plot_unstructured_image(images_snr1, images_snr2, images_scaled_SNR1, images_scaled_SNR2, a_U_snr1, a_U_snr2, index=0):
    # Recover the original images by subtracting the Gaussian noise
    original_image_snr1 = images_snr1[index] - a_U_snr1[index]
    original_image_snr2 = images_snr2[index] - a_U_snr2[index]
    
    original_image_scaled_snr1 = images_scaled_SNR1[index]
    original_image_scaled_snr2 = images_scaled_SNR2[index]
    
    # var_diff = variance_of_difference(original_image_snr1, original_image_snr2)
    # mae = mean_absolute_error(original_image_snr1, original_image_snr2)
    # pixel_diff = pixel_difference_count(original_image_snr1, original_image_snr2)
    # logging.info(f"Variance={var_diff:.4f}\nMAE={mae:.4f}\nPixel Diff={pixel_diff}")
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    # Plot original images
    axes[0, 0].imshow(original_image_snr1, cmap="gray")
    axes[0, 0].set_title(f"Original Image (SNR=1)\n")
    axes[0, 0].axis('off')

    axes[0, 1].imshow(original_image_snr2, cmap="gray")
    axes[0, 1].set_title(f"Original Image (SNR=2)\n")
    axes[0, 1].axis('off')

    # Plot noisy images
    axes[1, 0].imshow(original_image_scaled_snr1, cmap="gray")
    axes[1, 0].set_title(f"Noisy Image (SNR=1)\n")
    axes[1, 0].axis('off')

    axes[1, 1].imshow(original_image_scaled_snr2, cmap="gray")
    axes[1, 1].set_title(f"Noisy Image (SNR=2)\n")
    axes[1, 1].axis('off')

    plt.tight_layout()
    plt.show()
    
# def plot_linear_effects(X1, X2, linear_effects1, linear_effects2, scenario1, scenario2, K):
#     fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
    
#     for k in range(K):
#         for i in range(linear_effects1.shape[2]):  # Iterate over I
#             axes[0].plot(X1[i, :, i], linear_effects1[i, :, k], 'o', 
#                          label=f'Effect {k+1}, X{i+1} for low SNR', color="turquoise" if k == 0 else "violet")
#             axes[1].plot(X2[i, :, k], linear_effects2[i, :, k], 'o', 
#                          label=f'Effect {k+1}, X{i+1} for high SNR', color="orange" if k == 0 else "blue")
    
#     axes[0].set_title(f"Linear Effects (low SNR) - Scenario {scenario1}")
#     axes[0].set_xlabel("X (scaled)")
#     axes[0].set_ylabel("True Effect Value")
#     axes[0].legend()

#     axes[1].set_title(f"Linear Effects (high SNR) - Scenario {scenario2}")
#     axes[1].set_xlabel("X (scaled)")
#     axes[1].legend()

#     plt.tight_layout()
#     plt.show()

# def plot_nonlinear_effects(Z1, Z2, nonlinear_effects1, nonlinear_effects2, scenario1, scenario2, K):
#     fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
    
#     for k in range(K):
#         for j in range(nonlinear_effects1.shape[2]):  # Iterate over J
#             axes[0].plot(Z1[j, :, k], nonlinear_effects1[j, :, k], 'o', 
#                          label=f'Nonlinear Effect {k+1}, Z{j+1} for low SNR', color="turquoise" if k == 0 else "violet")
#             axes[1].plot(Z2[j, :, k], nonlinear_effects2[k, :, j], 'o', 
#                          label=f'Nonlinear Effect {k+1}, Z{j+1} for high SNR', color="orange" if k == 0 else "blue")
    
#     axes[0].set_title(f"Nonlinear Effects (low SNR) - Scenario {scenario1}")
#     axes[0].set_xlabel("Z (scaled)")
#     axes[0].set_ylabel("True Effect Value")
#     axes[0].legend()

#     axes[1].set_title(f"Nonlinear Effects (high SNR) - Scenario {scenario2}")
#     axes[1].set_xlabel("Z (scaled)")
#     axes[1].legend()

#     plt.tight_layout()
#     plt.show()

# def plot_unstructured_image(image_snr2, image_snr4, scenario):
#     var_diff = variance_of_difference(image_snr2, image_snr4)
#     mae = mean_absolute_error(image_snr2, image_snr4)
#     pixel_diff = pixel_difference_count(image_snr2, image_snr4)
#     logging.info(f"Variance={var_diff:.4f}\nMAE={mae:.4f}\nPixel Diff={pixel_diff}")
#     fig, axes = plt.subplots(1, 2, figsize=(12, 5))

#     axes[0].imshow(image_snr2, cmap="gray")
#     axes[0].set_title(f"Unstructured Effect (SNR=2)\n")
#     axes[0].axis('off')

#     axes[1].imshow(image_snr4, cmap="gray")
#     axes[1].set_title(f"Unstructured Effect (SNR=4)\n")
#     axes[1].axis('off')

#     plt.colorbar(axes[1].imshow(image_snr4, cmap="gray"))
#     plt.tight_layout()
#     plt.show()



# ---------------------------
# Generate Random Scenario for Visualization and Compare SNR
# ---------------------------

def random_scenario_plot(scenarios, alpha, beta, n_rep, SNR_compare=[0.5,20], grid_size=28):
    random_scenario = random.choice(scenarios)
    n_samples, distribution = random_scenario
    logging.info(f"Comparing SNR in {SNR_compare} for scenario: {random_scenario}")
    K = 2 if distribution in ["gamma", "gaussian"] else 1
   

    rep = random.randint(0, n_rep)
    scenario = '_'.join(map(str, random_scenario))+f"_rep{rep}"
    scenario_SNR1 = scenario+f"_SNR{SNR_compare[0]}"
    scenario_SNR2 = scenario+f"_SNR{SNR_compare[1]}"
    logging.info(f"Visualizing Scenario: {random_scenario}, Rep: {rep}")
    
    # Load the saved data from files
    # Define file paths
    dnn_model1 = dnn_model2 = build_dnn(grid_size * grid_size)
    dnn_model1.load_weights(f"{save_path}/dnn_model_{scenario_SNR1}.keras")
    dnn_model2.load_weights(f"{save_path}/dnn_model_{scenario_SNR2}.keras")
    
    # common basis
    X_path = f"{save_path}/X_{scenario}.npy"
    Z_path = f"{save_path}/Z_{scenario}.npy"
    images = f"{save_path}/images_{scenario}.npy"
    
    # lower SNR
    a_X_SNR1_path = f"{save_path}/a_X_{scenario_SNR1}.npy"
    a_Z_SNR1_path = f"{save_path}/a_Z_{scenario_SNR1}.npy"
    images_scaled_SNR1_path = f"{save_path}/images_scaled_{scenario_SNR1}.npy"
    a_U_SNR1_path = f"{save_path}/a_U_{scenario_SNR1}.npy"
    
    U_k_SNR1_path = f"{save_path}/U_k_{scenario_SNR1}.npy"
    psi_k_SNR1_path = f"{save_path}/psi_k_{scenario_SNR1}.npy"
    b_k_SNR1_path = f"{save_path}/b_k_{scenario_SNR1}.npy"
    
    etas_SNR1_path = f"{save_path}/etas_{scenario_SNR1}.npy"
    responses_SNR1_path = f"{save_path}/responses_{scenario_SNR1}.npy"
    
    linear_effects_SNR1_path = f"{save_path}/linear_effects_{scenario_SNR1}.npy"
    nonlinear_effects_SNR1_path = f"{save_path}/nonlinear_effects_{scenario_SNR1}.npy"
    unstructured_effects_SNR1_path = f"{save_path}/unstructured_effects_{scenario_SNR1}.npy"

    # higher SNR
    a_X_SNR2_path = f"{save_path}/a_X_{scenario_SNR2}.npy"
    a_Z_SNR2_path = f"{save_path}/a_Z_{scenario_SNR2}.npy"
    images_scaled_SNR2_path = f"{save_path}/images_scaled_{scenario_SNR2}.npy"
    a_U_SNR2_path = f"{save_path}/a_U_{scenario_SNR2}.npy"
    
    U_k_SNR2_path = f"{save_path}/U_k_{scenario_SNR2}.npy"
    psi_k_SNR2_path = f"{save_path}/psi_k_{scenario_SNR2}.npy"
    b_k_SNR2_path = f"{save_path}/b_k_{scenario_SNR2}.npy"
    
    etas_SNR2_path = f"{save_path}/etas_{scenario_SNR2}.npy"
    responses_SNR2_path = f"{save_path}/responses_{scenario_SNR2}.npy"
    
    linear_effects_SNR2_path = f"{save_path}/linear_effects_{scenario_SNR2}.npy"
    nonlinear_effects_SNR2_path = f"{save_path}/nonlinear_effects_{scenario_SNR2}.npy"
    unstructured_effects_SNR2_path = f"{save_path}/unstructured_effects_{scenario_SNR2}.npy"
    
    
    # Check if files exist
    files = [X_path,Z_path,images, a_X_SNR1_path, a_Z_SNR1_path, images_scaled_SNR1_path,
             a_U_SNR1_path, U_k_SNR1_path, psi_k_SNR1_path, b_k_SNR1_path, etas_SNR1_path, responses_SNR1_path,
             linear_effects_SNR1_path, nonlinear_effects_SNR1_path, unstructured_effects_SNR1_path,
             a_X_SNR2_path, a_Z_SNR2_path, images_scaled_SNR2_path,
             a_U_SNR2_path, U_k_SNR2_path,  psi_k_SNR2_path, b_k_SNR2_path, etas_SNR2_path, responses_SNR2_path,
             linear_effects_SNR2_path, nonlinear_effects_SNR2_path, unstructured_effects_SNR2_path]
    
    for file in files:
        if not os.path.exists(file):
            logging.warning(f"File not found: {file}")
            return False

    # Load the files
    X_SNR = np.load(X_path)
    Z_SNR = np.load(Z_path)
    
    # lower SNR
    a_X_SNR1 = np.load(a_X_SNR1_path)
    a_Z_SNR1 = np.load(a_Z_SNR1_path)
    images_scaled_SNR1 = np.load(images_scaled_SNR1_path)
    a_U_SNR1 = np.load(a_U_SNR1_path)
    U_k_SNR1 = np.load(U_k_SNR1_path)
    psi_k_SNR1 = np.load(psi_k_SNR1_path)
    b_k_SNR1 = np.load(b_k_SNR1_path)
    etas_SNR1 = np.load(etas_SNR1_path)
    responses_SNR1 = np.load(responses_SNR1_path)
    linear_effects_SNR1 = np.load(linear_effects_SNR1_path)
    print(linear_effects_SNR1)
    nonlinear_effects_SNR1 = np.load(nonlinear_effects_SNR1_path)
    unstructured_effects_SNR1 = np.load(unstructured_effects_SNR1_path)
    # higher SNR
    a_X_SNR2 = np.load(a_X_SNR2_path)
    a_Z_SNR2 = np.load(a_Z_SNR2_path)
    images_scaled_SNR2 = np.load(images_scaled_SNR2_path)
    a_U_SNR2 = np.load(a_U_SNR2_path)
    U_k_SNR2 = np.load(U_k_SNR2_path)
    psi_k_SNR2 = np.load(psi_k_SNR2_path)
    b_k_SNR2 = np.load(b_k_SNR2_path)
    etas_SNR2 = np.load(etas_SNR2_path)
    responses_SNR2 = np.load(responses_SNR2_path)
    linear_effects_SNR2 = np.load(linear_effects_SNR2_path)
    print(linear_effects_SNR2)
    nonlinear_effects_SNR2 = np.load(nonlinear_effects_SNR2_path)
    unstructured_effects_SNR2 = np.load(unstructured_effects_SNR2_path)
    
    # Replicate linear effects
    # Sanity checks for linear effects
    for i in range(linear_effects_SNR1.shape[0]):
        for k in range(linear_effects_SNR1.shape[2]):
            assert np.min(linear_effects_SNR1[i, :, k]) >= -1 and np.max(linear_effects_SNR1[i, :, k]) <= 1,\
                f"Linear effect column {i} for K={k} and SNR={SNR_compare[0]} is out of range!"
            assert np.min(linear_effects_SNR2[i, :, k]) >= -1 and np.max(linear_effects_SNR2[i, :, k]) <= 1,\
                f"Linear effect column {i} for K={k} and SNR={SNR_compare[1]} is out of range!"

    # Sanity checks for nonlinear effects
    for j in range(nonlinear_effects_SNR1.shape[2]):
        assert np.min(nonlinear_effects_SNR1[j, :, :]) >= -1 and np.max(nonlinear_effects_SNR1[j, :, :]) <= 1, f"Nonlinear effect column {j} for SNR=0.5 is out of range!"
        assert np.min(nonlinear_effects_SNR2[j, :, :]) >= -1 and np.max(nonlinear_effects_SNR2[j, :, :]) <= 1, f"Nonlinear effect column {j} for SNR=2 is out of range!"

    # Sanity checks for unstructured effects
    for k in range(unstructured_effects_SNR1.shape[1]):
        assert np.min(unstructured_effects_SNR1[:, k]) >= -1 and np.max(unstructured_effects_SNR1[:, k]) <= 1, f"Unstructured effect column {k} for SNR=0.5 is out of range!"
        assert np.min(unstructured_effects_SNR2[:, k]) >= -1 and np.max(unstructured_effects_SNR2[:, k]) <= 1, f"Unstructured effect column {k} for SNR=2 is out of range!"

    # # Generate unstructured image using Gaussian Process
    # x = np.linspace(0, 1, grid_size)
    # y = np.linspace(0, 1, grid_size)
    # X_grid, Y_grid = np.meshgrid(x, y)
    # coords = np.vstack([X_grid.ravel(), Y_grid.ravel()]).T

    # kernel = RBF(length_scale=0.2)
    # gp = GaussianProcessRegressor(kernel=kernel)
    # image_snr2 = gp.sample_y(coords, n_samples=1).reshape(grid_size, grid_size)
    # image_snr4 = image_snr2 * 2  # Simulate stronger signal for SNR=4
    # image_snr2 = scale_to_range(image_snr2, -1, 1)
    # image_snr4 = scale_to_range(image_snr4, -1, 1)
    # snr_image2 = calculate_snr(image_snr2)
    # snr_image4 = calculate_snr(image_snr4)
    # logging.info(f"SNR (Unstructured Image) - SNR=2: {snr_image2:.4f}, SNR=4: {snr_image4:.4f}")

    # Plot the results dynamically based on K
    plot_linear_effects(X_SNR, linear_effects_SNR1, linear_effects_SNR2, scenario_SNR1, scenario_SNR2, K)
    plot_nonlinear_effects(Z_SNR, nonlinear_effects_SNR1, nonlinear_effects_SNR2,scenario_SNR1,scenario_SNR2, K)
    # plot_unstructured_image(images_SNR1, images_SNR2, images_scaled_SNR1, images_scaled_SNR2, a_U_SNR1, a_U_SNR2, index=rep)



# ---------------------------
# Run Random Visualization for Comparison
# ---------------------------
# scenarios = [
#     (100, "poisson", 1, 2), (500, "poisson", 1, 2), (1000, "poisson", 1, 2),
#     (100, "poisson", 1, 4), (500, "poisson", 1, 4), (1000, "poisson", 1, 4),
#     (100, "gamma", 2, 2), (500, "gamma", 2, 2), (1000, "gamma", 2, 2), 
#     (100, "gamma", 2, 4), (500, "gamma", 2, 4), (1000, "gamma", 2, 4), 
#     (100, "gaussian", 2, 2), (500, "gaussian", 2, 2), (1000, "gaussian", 2, 2),
#     (100, "gaussian", 2, 4), (500, "gaussian", 2, 4), (1000, "gaussian", 2, 4)
# ]
scenarios = [
    (100, "poisson")#, (500, "poisson"), (1000, "poisson"),
    # (100, "gamma"), (500, "gamma"), (1000, "gamma"), 
    # (100, "gaussian"), (500, "gaussian"), (1000, "gaussian")
]

# Linear effect coefficients
alpha = {0: [3, -1], 1: [-0.5, 6]}

# Nonlinear effect functions
def nonlinear_effect_1_z1(z1):
    return scale_to_range(3 * np.sin(6 * z1))

def nonlinear_effect_1_z2(z2):
    return scale_to_range(np.exp(5 * z2))

def nonlinear_effect_2_z1(z1):
    return scale_to_range(np.cos(8 * z1))

def nonlinear_effect_2_z2(z2):
    return scale_to_range(0.1 * np.sqrt(z2))

beta = {
    0: [nonlinear_effect_1_z1, nonlinear_effect_1_z2],
    1: [nonlinear_effect_2_z1, nonlinear_effect_2_z2]
}

random_scenario_plot(scenarios, alpha, beta, n_rep=0, SNR_compare=[2,20],grid_size=28)
